In [3]:
# https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [11]:
# lele construct the training datasets



def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

# ix_to_tag = {v:k for k,v in tag_to_ix.items()}
# definition
id_to_nodes = {"Af":"FileScan FactTable", "Ad":"FileScan DimTable", 
               "B":"BroadcastHashJoin", "C":"SortMergeJoin", "D":"Union", "E": "End"}

id_to_operators = {"":"", "1":"Filter", "2":"Project", 
                   "3":"HashAggregate", "4":"Sort", 
                   "5":"TakeOrderedAndProject","6":"Expand","7":"Window","8":"SortAggregate"}


EMBEDDING_DIM = 6
HIDDEN_DIM = 6
EPOCHES = 500

training_data = [
    ("AfB BE".split(), ["1", "2-4-7-4-7-4-7-4-7-4-7-4-7"]),  # app1-a-plan
#     ("AfB BE".split(), ["1-2", "2-4-7-4-7-4-7-4-7-4-7-4-7"]),  # app1-a-plan

    ("AfE".split(), ["1-2-4-8-4-8-8-4-8"]),  # app1-b-plan

#     ("AfE".split(), ["1-2-4-8-4-8-8-4-8"]),  # app1-query28

#     ("AfC CC CC CC CC CE".split(), ["4", "2", "2", "2", "2", "2"]),  # app1-query30
# #     ("AfC CC CC CC CC CE".split(), ["1-4", "2", "2", "2", "2", "2"]),  # app1-query30
#     ("AfC CC CC CC CE".split(), ["1-4", "2", "2", "2", "2"]),  # app1-query30
#     ("AfC CC CC CE".split(), ["1-4", "2", "2", "2"]),  # app1-query30
#     ("AfC CC CE".split(), ["1-4", "2", "2"]),  # app1-query30
#     ("AfC CE".split(), ["1-4", "2"]),  # app1-query30

    ("AfE".split(), ["1-2"]),  # app3-a-plan

#     ("AfE".split(), ["1-2"]),  # app5-a-plan

    ("AfD DB BB BE".split(), ["1-2", "", "2", "2"]),  # app5-b-plan
#     ("AfD DB BB BE".split(), ["1-2", "", "2", "2"]),  # app5-b-plan
    ("AfB BE".split(), ["1", "2"]),  # app5-b-plan
    ("AfB BD DC CE".split(), ["1-2-1-2", "1", "4-8-4-8-4", "2-3-3"]),  # app6-a-plan
    ("AfD DC CE".split(), ["1-2", "4-8-4-8-4", "2-3-3"]),  # app6-a-plan
    ("AfC CE".split(), ["1-2", "2-3-3"]),  # app6-a-plan

    ("AfD DE".split(), ["1-2-6-3-3-4-7-1-2", ""]),  # app8-a-plan
#     ("AfD DE".split(), ["1-2-6-3-3-4-7-1-2", ""]),  # app8-a-plan
#     ("AfD DE".split(), ["1-2-6-3-3-4-7-1-2", ""]),  # app8-a-plan

    ("AfC CE".split(), ["1-2-3-4", "2"]),  # app10-a-plan
    ("AfC CC CE".split(), ["1-2-4-8-4-8-1-2-4", "2", "2"]),  # app10-a-plan
    ("AfC CC CE".split(), ["1-2-4-8-4-8", "", "2"]),  # app10-a-plan

    ("AfB BC CC CE".split(), ["1-2-3", "2-3", "2-3-4", "2-4-8-4-8"]),  # app11-a-plan
#     ("AfB BC CC CE".split(), ["1-2-3", "2-3", "2-3-4", "2-4-8-4-8"]),  # app11-a-plan
    ("AfC CC CE".split(), ["1-2-3-4", "2-3-4", "2-4-8-4-8"]),  # app11-a-plan
    ("AfC CE".split(), ["1-2-4", "2-4-8-4-8"]),  # app11-a-plan

    ("AfE".split(), ["2"]),  # app23-a-plan
#     ("AfE".split(), ["2"]),  # app25-a-plan
    ("AfE".split(), ["1-2"]),  # app26-a-plan
    

]
word_to_ix = {}
# For each words-list (sentence) and tags-list in each tuple of training_data
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:  # word has not been assigned an index yet
            word_to_ix[word] = len(word_to_ix)  # Assign each word with a unique index
print(word_to_ix)

tag_to_ix={}
for sent, tags in training_data:
    for tag in tags:
        if tag not in tag_to_ix:  # tag has not been assigned an index yet
            tag_to_ix[tag] = len(tag_to_ix)  # Assign each tag with a unique index
# print(tag_to_ix)

ix_to_tag = {v:k for k,v in tag_to_ix.items()}
# print(ix_to_tag)


# tag_to_ix = {"DET": 0, "NN": 1, "V": 2}  # Assign each tag with a unique index



# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.

# EPOCHES = 200, accuracy 0.625

# EPOCHES = 300, accuracy 0.625
# EPOCHES = 500, accuracy 0.625

{'AfB': 0, 'BE': 1, 'AfE': 2, 'AfD': 3, 'DB': 4, 'BB': 5, 'BD': 6, 'DC': 7, 'CE': 8, 'AfC': 9, 'DE': 10, 'CC': 11, 'BC': 12}


In [12]:

frequency_map={}
# for sent, tags in training_data:
#     for word in sent:
#         if word not in frequency_map:  # word has not been assigned an index yet
#             frequency_map[word] = len(word_to_ix)  # Assign each word with a unique index

for sent, tags in training_data:
    for (join_word, seq) in zip(sent, tags):
#         print(join_word, seq)
        if join_word not in frequency_map:
            frequency_map[join_word] = {}
        
        word_dict = frequency_map[join_word]
            
        if seq not in word_dict:
            word_dict[seq] = 1
        else:
            word_dict[seq] = word_dict[seq] +1

    
for word, word_dict in frequency_map.items():
    print("-------------")

#     print("word is {}".format(word))
    part1=""
    if word[0]=="A":
        part1=id_to_nodes[word[0:2]]

    else:
        part1=id_to_nodes[word[0]]
    part2 = id_to_nodes[word[-1]]
    converted_word = part1+"-"+part2
    print("converted_word is {}".format(converted_word))
    for seq_t, cnt in word_dict.items():
#         print("seq_t is {}, cnt is {}".format(seq_t, cnt))
        
        temp_list = seq_t.split("-")
        temp_ans = []
        for key in temp_list:
            temp_ans.append(id_to_operators[key])
#         print("temp_ans is {}".format(temp_ans))
        operator_list_str = "-".join(temp_ans)
        print("operator_list_str is {}, cnt is {}".format(operator_list_str, cnt))

-------------
converted_word is FileScan FactTable-BroadcastHashJoin
operator_list_str is Filter, cnt is 2
operator_list_str is Filter-Project-Filter-Project, cnt is 1
operator_list_str is Filter-Project-HashAggregate, cnt is 1
-------------
converted_word is BroadcastHashJoin-End
operator_list_str is Project-Sort-Window-Sort-Window-Sort-Window-Sort-Window-Sort-Window-Sort-Window, cnt is 1
operator_list_str is Project, cnt is 2
-------------
converted_word is FileScan FactTable-End
operator_list_str is Filter-Project-Sort-SortAggregate-Sort-SortAggregate-SortAggregate-Sort-SortAggregate, cnt is 1
operator_list_str is Filter-Project, cnt is 2
operator_list_str is Project, cnt is 1
-------------
converted_word is FileScan FactTable-Union
operator_list_str is Filter-Project, cnt is 2
operator_list_str is Filter-Project-Expand-HashAggregate-HashAggregate-Sort-Window-Filter-Project, cnt is 1
-------------
converted_word is Union-BroadcastHashJoin
operator_list_str is , cnt is 1
------------

In [2]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence),-1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores
    


model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
#     print(tag_scores)

for epoch in range(EPOCHES):  # again, normally you would NOT do 300 epochs, it is toy data
    item_index = 1
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)
#         print("item_index  is {}， sentence is {}".format(item_index, sentence))
        item_index = item_index +1
#         print("sentence_in.shape is {}".format(sentence_in.shape))
#         print("tag_scores.shape is {}".format(tag_scores.shape))

#         print("targets.shape is {}".format(targets.shape))

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    # {'1': 0, '2-4': 1, '2': 2, '2-3-3-1-5': 3, '2-3-3-5': 4}
#     print(tag_scores)
#     print(torch.max(tag_scores,1))


    
# 
correct_predictions = 0
total_predictions = 0
for sentence, tags in training_data:
    # Prepare the sentence for the model
    sentence_in = prepare_sequence(sentence, word_to_ix)
    
    # Get predicted tag indices
    predicted_tags = torch.argmax(model(sentence_in), dim=1)
    
    # Convert tensor to numpy array
    predicted_tags = predicted_tags.numpy()
    
    # Convert ground truth tags to indices
    true_tags = [tag_to_ix[tag] for tag in tags]
    
    # Calculate accuracy for this sentence
    correct_predictions += sum(predicted_tag == true_tag for predicted_tag, true_tag in zip(predicted_tags, true_tags))
    total_predictions += len(tags)

accuracy = correct_predictions / total_predictions
print("Accuracy:", accuracy)
    

NameError: name 'nn' is not defined

-------------
converted_word is FileScan FactTable-BroadcastHashJoin
operator_list_str is Filter, cnt is 1
operator_list_str is Filter-Project-Filter-Project, cnt is 1
operator_list_str is Filter-Project-HashAggregate, cnt is 1
-------------
converted_word is BroadcastHashJoin-
operator_list_str is Project, cnt is 2
-------------
converted_word is FileScan FactTable-
operator_list_str is Filter-Project, cnt is 2
operator_list_str is Project, cnt is 1
-------------
converted_word is FileScan FactTable-Union
operator_list_str is Filter-Project, cnt is 1
operator_list_str is Filter-Project-Expand-HashAggregate-HashAggregate-Sort-Window-Filter-Project, cnt is 1
-------------
converted_word is Union-BroadcastHashJoin
-------------
converted_word is BroadcastHashJoin-BroadcastHashJoin
-------------
converted_word is BroadcastHashJoin-Union
-------------
converted_word is Union-SortMergeJoin
operator_list_str is Sort-SortAggregate-Sort-SortAggregate-Sort, cnt is 1
-------------
converted_word

In [75]:
# test_seq="AfB BC CC CE".split()
# test_seq="AfB BE".split()
# test_seq="CB BE".split()
# test_seq="AfC CE".split()
# test_seq="AfC CC CE".split()

test_data = [
    "AfC CC CC CE".split(),
    "AfC CE".split(),
    "AfE".split(),
    "AfB BE".split()
]
# test_seq="AfC CC CC CE".split()


# test_seq="AfC CE".split()

# test_seq="AE".split()
for i,test_seq in enumerate(test_data):
    print(" ---- index is {} ------------".format(i))
    inputs = prepare_sequence(test_seq, word_to_ix)
    tag_scores = model(inputs)
    # print(tag_scores)

    ans = torch.max(tag_scores,1)
    ans_list = ans.indices.numpy().tolist()
    print(ans_list)
    for item in ans_list:
        temp_str = ix_to_tag[item]
        temp_list = temp_str.split("-")
        temp_ans = []
        for key in temp_list:
            temp_ans.append(id_to_operators[key])
        print("-".join(temp_ans))

 ---- index is 0 ------------
[10, 15, 15, 16]
Filter-Project-HashAggregate-Sort
Project-HashAggregate-Sort
Project-HashAggregate-Sort
Project-Sort-SortAggregate-Sort-SortAggregate
 ---- index is 1 ------------
[10, 16]
Filter-Project-HashAggregate-Sort
Project-Sort-SortAggregate-Sort-SortAggregate
 ---- index is 2 ------------
[3]
Filter-Project
 ---- index is 3 ------------
[0, 5]
Filter
Project
